In [1]:
# Loading up libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
# Loading up csv
# df = pd.read_csv(r"./csv/data_3600.csv")
# df = pd.read_csv(r"./csv/data_36000.csv")
df = pd.read_csv(r"./csv/data_all.csv")
pd.set_option('display.max_rows', 500)

In [3]:
#2,22 : 6,11
SMAL = df['Price'].rolling(window = 2).mean()
SMAH = df['Price'].rolling(window = 22).mean()
df["SMAL"] = SMAL
df["SMAH"] = SMAH

In [4]:
def checkCrossover(row):
    if row["SMAL"] > row["SMAH"]:
        return "Higher"
    else:
        return "Lower"

def timeFormatter(mins):
    time_hr = mins//60
    time_min = mins%60
    
    time_output = str(time_hr).zfill(4) + ':' + str(time_min).zfill(2) + ':00'
    return time_output

def reportMessage(report, _time, added_msg=""):
    buy_time = timeFormatter(report["buy"]["time"])
    buy_price = round(report["buy"]["price"],4)
    sell_time = timeFormatter(report["sell"]["time"])
    sell_price = round(report["sell"]["price"],4)
    diff_price = round(sell_price-buy_price,4)
    print(f'[{buy_time}] Open({buy_price:.4f}) | [{sell_time}] Close({sell_price:.4f}) | Profit ({diff_price:.4f}) <{_time}mins> | {added_msg}')
    

In [5]:
df["Crossover"] = df.apply(checkCrossover, axis=1)

In [6]:
df["Change"] = ""
df["isBuy"] = ""
df["TimeLimit"] = 0
df["Action"] = "---"
df["CurrentPrice"] = "---"

In [7]:

report = {
    "buy":{
        "time":0,
        "price":0
    },
    "sell":{
        "time":0,
        "price":0
    },
    "profit": 0
}

for index, row in df.iterrows():
    if index == 0:
        df["Change"][index] = "N/A"
        df["isBuy"][index] = True
        df["TimeLimit"][index] = 0
        df["Action"][index] = "Buy!"
        df["CurrentPrice"][index] = float(df["Price"][index])
        __currentMode = "Buy!"
        __currentProfit = df["CurrentPrice"][index]
        report["buy"]["price"] = df["Price"][index]


    else:
            df["TimeLimit"][index] = df["TimeLimit"][index - 1] + 1
        
            __crossOverPrev = df["Crossover"][index - 1]
            __crossOverPres = df["Crossover"][index]
            __timeLimit = df["TimeLimit"][index]

            if __crossOverPrev != __crossOverPres:
                df["Change"][index] = "Crossed"
            else:
                df["Change"][index] = "---"
            
            
            if df["Crossover"][index] == "Lower":
                df["isBuy"][index] = True
            else:
                df["isBuy"][index] = False           
            
            
            
            df["TimeLimit"][index] = __timeLimit                        
                    
            if df["Change"][index] == "Crossed" and df["isBuy"][index] and __currentMode != "Buy!":
                df["Action"][index] = "Buy!"
                __currentMode = "Buy!"
                df["CurrentPrice"][index] = df["Price"][index]
                report["buy"]["time"] = df["Time"][index]
                report["buy"]["price"] = df["Price"][index]
                df["TimeLimit"][index] = 0 
                
#             if  df["Price"][index] < report["buy"]["price"] and df["Change"][index] == "Crossed" and __currentMode == "Buy!":
#                 df["Change"][index] == "Skip"
#                 df["Change"][index + 1] == "Crossed"
                
            if  df["Price"][index] > report["buy"]["price"] and df["TimeLimit"][index] in range(30,59 + 1) and df["Change"][index] == "Crossed" and not df["isBuy"][index] and __currentMode != "Sell!":
                df["Action"][index] = "Sell!"
                __currentMode = "Sell!"                    
                df["CurrentPrice"][index] = df["Price"][index]
                report["sell"]["time"] = df["Time"][index]
                report["sell"]["price"] = df["Price"][index]
                report["profit"] +=  report["sell"]["price"] -  report["buy"]["price"]                    
                reportMessage(report, df["TimeLimit"][index])

            
            elif df["TimeLimit"][index] in range(57,59) and df["Price"][index] > report["buy"]["price"] and __currentMode != "Sell!":
                df["Action"][index] = "Sell!"
                __currentMode = "Sell!"
                df["CurrentPrice"][index] = df["Price"][index]
                report["sell"]["time"] = df["Time"][index]
                report["sell"]["price"] = df["Price"][index]
                report["profit"] +=  report["sell"]["price"] -  report["buy"]["price"]
                reportMessage(report, df["TimeLimit"][index],"CLOSED")


            
            elif df["TimeLimit"][index] >= 59 and __currentMode == "Buy!":
                df["Action"][index] = "Sell!"
                __currentMode = "Sell!"
                df["CurrentPrice"][index] = df["Price"][index]
                report["sell"]["time"] = df["Time"][index]
                report["sell"]["price"] = df["Price"][index]
                report["profit"] +=  report["sell"]["price"] -  report["buy"]["price"]
                reportMessage(report, df["TimeLimit"][index],"FORCED")
                            
print(f'\nTotal profit: {round(report["profit"],4):.4f}')

[0000:00:00] Open(1.2546) | [0000:46:00] Close(1.2629) | Profit (0.0083) <46mins> | 
[0001:04:00] Open(1.2635) | [0001:46:00] Close(1.2832) | Profit (0.0197) <42mins> | 
[0001:47:00] Open(1.2769) | [0002:17:00] Close(1.2914) | Profit (0.0145) <30mins> | 
[0002:19:00] Open(1.2846) | [0002:55:00] Close(1.3001) | Profit (0.0155) <36mins> | 
[0002:57:00] Open(1.2966) | [0003:54:00] Close(1.3665) | Profit (0.0699) <57mins> | CLOSED
[0004:07:00] Open(1.3574) | [0004:38:00] Close(1.3648) | Profit (0.0074) <31mins> | 
[0004:40:00] Open(1.3619) | [0005:39:00] Close(1.3583) | Profit (-0.0036) <59mins> | FORCED
[0005:41:00] Open(1.3580) | [0006:40:00] Close(1.3201) | Profit (-0.0379) <59mins> | FORCED
[0006:42:00] Open(1.3188) | [0007:17:00] Close(1.3257) | Profit (0.0069) <35mins> | 
[0007:22:00] Open(1.3221) | [0008:03:00] Close(1.3460) | Profit (0.0239) <41mins> | 
[0008:05:00] Open(1.3430) | [0009:04:00] Close(1.3281) | Profit (-0.0149) <59mins> | FORCED
[0009:41:00] Open(1.3242) | [0010:38:0

[0103:14:00] Open(1.3590) | [0103:51:00] Close(1.3656) | Profit (0.0066) <37mins> | 
[0103:57:00] Open(1.3637) | [0104:54:00] Close(1.3669) | Profit (0.0032) <57mins> | CLOSED
[0105:01:00] Open(1.3664) | [0106:00:00] Close(1.3619) | Profit (-0.0045) <59mins> | FORCED
[0106:39:00] Open(1.3528) | [0107:38:00] Close(1.3323) | Profit (-0.0205) <59mins> | FORCED
[0107:58:00] Open(1.3301) | [0108:46:00] Close(1.3463) | Profit (0.0162) <48mins> | 
[0109:02:00] Open(1.3477) | [0109:54:00] Close(1.3537) | Profit (0.0060) <52mins> | 
[0110:12:00] Open(1.3550) | [0111:11:00] Close(1.3398) | Profit (-0.0152) <59mins> | FORCED
[0111:25:00] Open(1.3351) | [0112:22:00] Close(1.3493) | Profit (0.0142) <57mins> | CLOSED
[0112:38:00] Open(1.3584) | [0113:26:00] Close(1.3636) | Profit (0.0052) <48mins> | 
[0113:28:00] Open(1.3626) | [0114:27:00] Close(1.3461) | Profit (-0.0165) <59mins> | FORCED
[0114:55:00] Open(1.3466) | [0115:54:00] Close(1.3321) | Profit (-0.0145) <59mins> | FORCED
[0116:33:00] Open(

[0219:27:00] Open(1.4538) | [0219:59:00] Close(1.4560) | Profit (0.0022) <32mins> | 
[0220:04:00] Open(1.4545) | [0221:01:00] Close(1.4673) | Profit (0.0128) <57mins> | CLOSED
[0221:04:00] Open(1.4639) | [0221:36:00] Close(1.4842) | Profit (0.0203) <32mins> | 
[0221:39:00] Open(1.4822) | [0222:11:00] Close(1.4840) | Profit (0.0018) <32mins> | 
[0222:22:00] Open(1.4829) | [0223:18:00] Close(1.4861) | Profit (0.0032) <56mins> | 
[0223:24:00] Open(1.4830) | [0224:23:00] Close(1.4374) | Profit (-0.0456) <59mins> | FORCED
[0224:25:00] Open(1.4330) | [0225:24:00] Close(1.4169) | Profit (-0.0161) <59mins> | FORCED
[0225:38:00] Open(1.4197) | [0226:35:00] Close(1.4203) | Profit (0.0006) <57mins> | 
[0226:37:00] Open(1.4141) | [0227:11:00] Close(1.4253) | Profit (0.0112) <34mins> | 
[0227:21:00] Open(1.4219) | [0228:20:00] Close(1.4046) | Profit (-0.0173) <59mins> | FORCED
[0228:23:00] Open(1.4012) | [0229:00:00] Close(1.4072) | Profit (0.0060) <37mins> | 
[0229:01:00] Open(1.4052) | [0229:36:0

[0324:36:00] Open(1.3208) | [0325:18:00] Close(1.3224) | Profit (0.0016) <42mins> | 
[0325:41:00] Open(1.3239) | [0326:40:00] Close(1.3154) | Profit (-0.0085) <59mins> | FORCED
[0327:01:00] Open(1.3145) | [0328:00:00] Close(1.3029) | Profit (-0.0116) <59mins> | FORCED
[0328:16:00] Open(1.3011) | [0329:15:00] Close(1.2921) | Profit (-0.0090) <59mins> | FORCED
[0329:19:00] Open(1.2934) | [0330:18:00] Close(1.2734) | Profit (-0.0200) <59mins> | FORCED
[0330:39:00] Open(1.2722) | [0331:36:00] Close(1.2794) | Profit (0.0072) <57mins> | CLOSED
[0331:44:00] Open(1.2755) | [0332:43:00] Close(1.2594) | Profit (-0.0161) <59mins> | FORCED
[0333:01:00] Open(1.2543) | [0334:00:00] Close(1.2534) | Profit (-0.0009) <59mins> | FORCED
[0334:07:00] Open(1.2536) | [0335:06:00] Close(1.2359) | Profit (-0.0177) <59mins> | FORCED
[0336:04:00] Open(1.2447) | [0336:34:00] Close(1.2510) | Profit (0.0063) <30mins> | 
[0337:00:00] Open(1.2566) | [0337:59:00] Close(1.2482) | Profit (-0.0084) <59mins> | FORCED
[03

[0433:35:00] Open(1.2881) | [0434:34:00] Close(1.2868) | Profit (-0.0013) <59mins> | FORCED
[0435:00:00] Open(1.2878) | [0435:53:00] Close(1.2935) | Profit (0.0057) <53mins> | 
[0435:57:00] Open(1.2922) | [0436:31:00] Close(1.2929) | Profit (0.0007) <34mins> | 
[0436:35:00] Open(1.2856) | [0437:32:00] Close(1.3042) | Profit (0.0186) <57mins> | CLOSED
[0437:41:00] Open(1.3017) | [0438:12:00] Close(1.3097) | Profit (0.0080) <31mins> | 
[0438:34:00] Open(1.3066) | [0439:31:00] Close(1.3239) | Profit (0.0173) <57mins> | CLOSED
[0439:46:00] Open(1.3203) | [0440:36:00] Close(1.3273) | Profit (0.0070) <50mins> | 
[0440:55:00] Open(1.3279) | [0441:28:00] Close(1.3345) | Profit (0.0066) <33mins> | 
[0441:40:00] Open(1.3324) | [0442:14:00] Close(1.3341) | Profit (0.0017) <34mins> | 
[0442:25:00] Open(1.3349) | [0442:55:00] Close(1.3400) | Profit (0.0051) <30mins> | 
[0442:57:00] Open(1.3390) | [0443:56:00] Close(1.3105) | Profit (-0.0285) <59mins> | FORCED
[0444:31:00] Open(1.3095) | [0445:30:00

...

[0642:27:00] Open(1.0956) | [0643:24:00] Close(1.1129) | Profit (0.0173) <57mins> | CLOSED
[0643:54:00] Open(1.1283) | [0644:53:00] Close(1.1319) | Profit (0.0036) <59mins> | FORCED
[0645:02:00] Open(1.1265) | [0645:59:00] Close(1.1270) | Profit (0.0005) <57mins> | CLOSED
[0646:08:00] Open(1.1231) | [0646:41:00] Close(1.1232) | Profit (0.0001) <33mins> | 
[0647:01:00] Open(1.1251) | [0647:33:00] Close(1.1263) | Profit (0.0012) <32mins> | 
[0647:57:00] Open(1.1365) | [0648:41:00] Close(1.1369) | Profit (0.0004) <44mins> | 
[0648:45:00] Open(1.1342) | [0649:44:00] Close(1.1207) | Profit (-0.0135) <59mins> | FORCED
[0649:46:00] Open(1.1192) | [0650:44:00] Close(1.1214) | Profit (0.0022) <58mins> | CLOSED
[0650:58:00] Open(1.1164) | [0651:57:00] Close(1.1085) | Profit (-0.0079) <59mins> | FORCED
[0652:20:00] Open(1.1072) | [0653:19:00] Close(1.0990) | Profit (-0.0082) <59mins> | FORCED
[0653:48:00] Open(1.1044) | [0654:34:00] Close(1.1186) | Profit (0.0142) <46mins> | 
[0654:35:00] Open(1.

[0750:58:00] Open(1.1229) | [0751:50:00] Close(1.1338) | Profit (0.0109) <52mins> | 
[0752:00:00] Open(1.1311) | [0752:57:00] Close(1.1384) | Profit (0.0073) <57mins> | CLOSED
[0753:00:00] Open(1.1333) | [0753:59:00] Close(1.1089) | Profit (-0.0244) <59mins> | FORCED
[0754:06:00] Open(1.1040) | [0754:44:00] Close(1.1115) | Profit (0.0075) <38mins> | 
[0754:49:00] Open(1.1112) | [0755:37:00] Close(1.1144) | Profit (0.0032) <48mins> | 
[0755:50:00] Open(1.1127) | [0756:49:00] Close(1.1087) | Profit (-0.0040) <59mins> | FORCED
[0756:51:00] Open(1.1071) | [0757:50:00] Close(1.1077) | Profit (0.0006) <59mins> | FORCED
[0758:04:00] Open(1.1073) | [0758:53:00] Close(1.1113) | Profit (0.0040) <49mins> | 
[0759:00:00] Open(1.1106) | [0759:59:00] Close(1.1058) | Profit (-0.0048) <59mins> | FORCED
[0760:25:00] Open(1.0995) | [0760:57:00] Close(1.1023) | Profit (0.0028) <32mins> | 
[0761:12:00] Open(1.1024) | [0762:11:00] Close(1.0988) | Profit (-0.0036) <59mins> | FORCED
[0762:28:00] Open(1.0987)

[0853:29:00] Open(1.1203) | [0854:15:00] Close(1.1263) | Profit (0.0060) <46mins> | 
[0854:22:00] Open(1.1218) | [0855:21:00] Close(1.1204) | Profit (-0.0014) <59mins> | FORCED
[0855:24:00] Open(1.1192) | [0856:23:00] Close(1.1168) | Profit (-0.0024) <59mins> | FORCED
[0856:39:00] Open(1.1161) | [0857:27:00] Close(1.1211) | Profit (0.0050) <48mins> | 
[0857:31:00] Open(1.1187) | [0858:17:00] Close(1.1363) | Profit (0.0176) <46mins> | 
[0858:57:00] Open(1.1423) | [0859:56:00] Close(1.1351) | Profit (-0.0072) <59mins> | FORCED
[0860:12:00] Open(1.1324) | [0861:09:00] Close(1.1402) | Profit (0.0078) <57mins> | CLOSED
[0861:18:00] Open(1.1404) | [0862:15:00] Close(1.1467) | Profit (0.0063) <57mins> | CLOSED
[0862:35:00] Open(1.1403) | [0863:26:00] Close(1.1464) | Profit (0.0061) <51mins> | 
[0864:02:00] Open(1.1548) | [0864:39:00] Close(1.1562) | Profit (0.0014) <37mins> | 
[0865:21:00] Open(1.1646) | [0865:56:00] Close(1.1658) | Profit (0.0012) <35mins> | 
[0865:57:00] Open(1.1650) | [086

[0966:53:00] Open(1.1705) | [0967:52:00] Close(1.1645) | Profit (-0.0060) <59mins> | FORCED
[0968:26:00] Open(1.1706) | [0969:25:00] Close(1.1731) | Profit (0.0025) <59mins> | FORCED
[0969:39:00] Open(1.1714) | [0970:38:00] Close(1.1703) | Profit (-0.0011) <59mins> | FORCED
[0970:56:00] Open(1.1701) | [0971:55:00] Close(1.1583) | Profit (-0.0118) <59mins> | FORCED
[0972:07:00] Open(1.1572) | [0973:04:00] Close(1.1599) | Profit (0.0027) <57mins> | CLOSED
[0973:14:00] Open(1.1588) | [0974:13:00] Close(1.1401) | Profit (-0.0187) <59mins> | FORCED
[0974:50:00] Open(1.1385) | [0975:49:00] Close(1.1372) | Profit (-0.0013) <59mins> | FORCED
[0975:52:00] Open(1.1373) | [0976:31:00] Close(1.1428) | Profit (0.0055) <39mins> | 
[0977:30:00] Open(1.1566) | [0978:27:00] Close(1.1617) | Profit (0.0051) <57mins> | CLOSED
[0979:01:00] Open(1.1692) | [0980:00:00] Close(1.1672) | Profit (-0.0020) <59mins> | FORCED
[0980:05:00] Open(1.1651) | [0981:04:00] Close(1.1582) | Profit (-0.0069) <59mins> | FORCE

[1076:18:00] Open(1.1198) | [1076:55:00] Close(1.1204) | Profit (0.0006) <37mins> | 
[1077:07:00] Open(1.1201) | [1077:44:00] Close(1.1216) | Profit (0.0015) <37mins> | 
[1077:47:00] Open(1.1184) | [1078:46:00] Close(1.1104) | Profit (-0.0080) <59mins> | FORCED
[1079:02:00] Open(1.1093) | [1079:59:00] Close(1.1098) | Profit (0.0005) <57mins> | CLOSED
[1080:05:00] Open(1.1093) | [1080:35:00] Close(1.1096) | Profit (0.0003) <30mins> | 
[1080:37:00] Open(1.1089) | [1081:34:00] Close(1.1104) | Profit (0.0015) <57mins> | CLOSED
[1081:45:00] Open(1.1101) | [1082:44:00] Close(1.1058) | Profit (-0.0043) <59mins> | FORCED

Total profit: 0.1284


In [8]:
df[:250]

,Time,Price,SMAL,SMAH,Crossover,Change,isBuy,TimeLimit,Action,CurrentPrice
0,0,1.2546,NaN,NaN,Lower,N/A,True,0,Buy!,1.2546
1,1,1.2567,1.25565,NaN,Lower,---,True,1,---,---
2,2,1.2577,1.25720,NaN,Lower,---,True,2,---,---
3,3,1.2579,1.25780,NaN,Lower,---,True,3,---,---
4,4,1.2593,1.25860,NaN,Lower,---,True,4,---,---
5,5,1.2668,1.26305,NaN,Lower,---,True,5,---,---
6,6,1.2695,1.26815,NaN,Lower,---,True,6,---,---
7,7,1.2689,1.26920,NaN,Lower,---,True,7,---,---
8,8,1.2679,1.26840,NaN,Lower,---,True,8,---,---
9,9,1.2679,1.26790,NaN,Lower,---,True,9,---,---


In [9]:
buy = df[df["Action"] == "Buy!"]["Price"].sum()
df[df["Action"] == "Buy!"]["Price"].count()


943

In [10]:
sell = df[df["Action"] == "Sell!"]["Price"].sum()
df[df["Action"] == "Sell!"]["Price"].count()

943

In [11]:
sell - buy

0.12842000000000553